In [5]:
from google.colab import drive
# Step 1: Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
from collections import defaultdict
from google.colab.patches import cv2_imshow  # Import for Colab compatibility

# Dataset paths
dataset_root = "/content/drive/MyDrive/TharuWorks/Dataset/train"  # Update as needed
images_dir = os.path.join(dataset_root, "images")
labels_dir = os.path.join(dataset_root, "depth_labels")

# Function to convert YOLO format (normalized) to pixel coordinates
def yolo_to_pixel(yolo_box, img_width, img_height):
    """
    Convert YOLO bounding box format (normalized values) to pixel coordinates.
    """
    class_id, x_center, y_center, width, height, depth = map(float, yolo_box)
    x_min = int((x_center - width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    x_max = int((x_center + width / 2) * img_width)
    y_max = int((y_center + height / 2) * img_height)

    return (x_min, y_min, x_max, y_max, depth, int(class_id))

# Function to read labels and extract bounding box info
def load_labels(label_path, img_width, img_height):
    """
    Load label file and extract bounding boxes.
    """
    objects = []
    with open(label_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 6:  # Ensure there are enough elements
                objects.append(yolo_to_pixel(parts, img_width, img_height))
    return objects

# Function to compute IoU (Intersection over Union)
def compute_iou(box1, box2):
    """
    Compute IoU between two bounding boxes.
    """
    x1_min, y1_min, x1_max, y1_max = box1[:4]
    x2_min, y2_min, x2_max, y2_max = box2[:4]

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Function to track objects across frames
def track_objects(previous_objects, current_objects):
    """
    Match objects in the current frame to those in the previous frame.
    """
    matched_objects = {}

    for curr_id, curr_obj in enumerate(current_objects):
        x_min, y_min, x_max, y_max, depth, class_id = curr_obj
        best_match_id = None
        best_iou = 0

        for prev_id, prev_obj in previous_objects.items():
            prev_x_min, prev_y_min, prev_x_max, prev_y_max, prev_depth, prev_class_id = prev_obj
            if class_id == prev_class_id:  # Match only same-class objects
                iou = compute_iou((x_min, y_min, x_max, y_max), (prev_x_min, prev_y_min, prev_x_max, prev_y_max))
                if iou > best_iou and iou > 0.5:  # IoU threshold
                    best_iou = iou
                    best_match_id = prev_id

        if best_match_id is not None:
            matched_objects[best_match_id] = (x_min, y_min, x_max, y_max, depth, class_id)
        else:
            matched_objects[len(previous_objects) + curr_id] = (x_min, y_min, x_max, y_max, depth, class_id)

    return matched_objects

# Process dataset
previous_objects = {}  # Store objects from previous frame
image_files = sorted(os.listdir(images_dir))  # Sort images by filename (sequence order)

for image_file in image_files:
    if not image_file.endswith(".jpg"):
        continue

    image_path = os.path.join(images_dir, image_file)
    label_path = os.path.join(labels_dir, image_file.replace(".jpg", ".txt"))

    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Error loading image {image_file}, skipping...")
        continue

    img_height, img_width, _ = image.shape  # Get image dimensions

    # Load corresponding label file
    if not os.path.exists(label_path):
        print(f"⚠️ No label found for {image_file}, skipping...")
        continue

    current_objects = load_labels(label_path, img_width, img_height)

    # Track objects
    tracked_objects = track_objects(previous_objects, current_objects)
    previous_objects = tracked_objects.copy()

    # Draw bounding boxes
    for obj_id, (x_min, y_min, x_max, y_max, depth, class_id) in tracked_objects.items():
        color = (0, 255, 0) if class_id == 22 else (255, 0, 0)  # Green for vehicle, red for pedestrian
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
        cv2.putText(image, f"ID: {obj_id}, Depth: {depth:.2f}m", (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show tracked image (Google Colab friendly)
    cv2_imshow(image)


In [ ]:
import os
import cv2
import numpy as np
from collections import defaultdict
from google.colab.patches import cv2_imshow  # Import for Colab compatibility

# Dataset paths
dataset_root = "/content/drive/MyDrive/TharuWorks/Dataset/v"  # Update as needed
images_dir = os.path.join(dataset_root, "images")
labels_dir = os.path.join(dataset_root, "depth_labels")

# Function to convert YOLO format (normalized) to pixel coordinates
def yolo_to_pixel(yolo_box, img_width, img_height):
    """
    Convert YOLO bounding box format (normalized values) to pixel coordinates.
    """
    class_id, x_center, y_center, width, height, depth = map(float, yolo_box)
    x_min = int((x_center - width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    x_max = int((x_center + width / 2) * img_width)
    y_max = int((y_center + height / 2) * img_height)

    return (x_min, y_min, x_max, y_max, depth, int(class_id))

# Function to read labels and extract bounding box info
def load_labels(label_path, img_width, img_height):
    """
    Load label file and extract bounding boxes.
    """
    objects = []
    with open(label_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 6:  # Ensure there are enough elements
                objects.append(yolo_to_pixel(parts, img_width, img_height))
    return objects

# Function to compute IoU (Intersection over Union)
def compute_iou(box1, box2):
    """
    Compute IoU between two bounding boxes.
    """
    x1_min, y1_min, x1_max, y1_max = box1[:4]
    x2_min, y2_min, x2_max, y2_max = box2[:4]

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Function to track objects across frames
def track_objects(previous_objects, current_objects):
    """
    Match objects in the current frame to those in the previous frame.
    """
    matched_objects = {}

    for curr_id, curr_obj in enumerate(current_objects):
        x_min, y_min, x_max, y_max, depth, class_id = curr_obj
        best_match_id = None
        best_iou = 0

        for prev_id, prev_obj in previous_objects.items():
            prev_x_min, prev_y_min, prev_x_max, prev_y_max, prev_depth, prev_class_id = prev_obj
            if class_id == prev_class_id:  # Match only same-class objects
                iou = compute_iou((x_min, y_min, x_max, y_max), (prev_x_min, prev_y_min, prev_x_max, prev_y_max))
                if iou > best_iou and iou > 0.5:  # IoU threshold
                    best_iou = iou
                    best_match_id = prev_id

        if best_match_id is not None:
            matched_objects[best_match_id] = (x_min, y_min, x_max, y_max, depth, class_id)
        else:
            matched_objects[len(previous_objects) + curr_id] = (x_min, y_min, x_max, y_max, depth, class_id)

    return matched_objects

# Process dataset
previous_objects = {}  # Store objects from previous frame
image_files = sorted(os.listdir(images_dir))  # Sort images by filename (sequence order)

for image_file in image_files:
    if not image_file.endswith(".jpg"):
        continue

    image_path = os.path.join(images_dir, image_file)
    label_path = os.path.join(labels_dir, image_file.replace(".jpg", ".txt"))

    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Error loading image {image_file}, skipping...")
        continue

    img_height, img_width, _ = image.shape  # Get image dimensions

    # Load corresponding label file
    if not os.path.exists(label_path):
        print(f"⚠️ No label found for {image_file}, skipping...")
        continue

    current_objects = load_labels(label_path, img_width, img_height)

    # Track objects
    tracked_objects = track_objects(previous_objects, current_objects)
    previous_objects = tracked_objects.copy()

    # Draw bounding boxes
    for obj_id, (x_min, y_min, x_max, y_max, depth, class_id) in tracked_objects.items():
        color = (0, 255, 0) if class_id == 22 else (255, 0, 0)  # Green for vehicle, red for pedestrian
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
        cv2.putText(image, f"ID: {obj_id}, Depth: {depth:.2f}m", (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show tracked image (Google Colab friendly)
    cv2_imshow(image)
